In [2]:
%reload_ext ishbook
import pandas as pd
import numpy as np
import datetime as dt
import time
import iql
import plus
import re
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pull in the affected Advertisers from IQL query provided by CSP
* Elaborate on the advid to get the advertiser name

In [ ]:
%%ish

affected_Accounts = from adconversion 1w today where hrefdomain = 'adcentral.indeed.com' dupe!=1 group by advid
elaborate(affected_Accounts.advid)
Output = None

## Create a column to generate a link to the adCard for each affected advid

In [3]:
affected_Accounts["Link to AdCard"] = 'https://adcentral.indeed.com/crm/history?aid=' + affected_Accounts["advid"].astype(str)

In [4]:
affected_Accounts.advid

,advid,,_advid,Link to AdCard
0,4240,1,WyzAnt Tutoring,https://adcentral.indeed.com/crm/history?aid=4240
1,4653,11,Sage Software,https://adcentral.indeed.com/crm/history?aid=4653
2,17574,1,Hairandbeautyjobs.com Ltd,https://adcentral.indeed.com/crm/history?aid=1...
3,19296,1,Centura Health,https://adcentral.indeed.com/crm/history?aid=1...
4,19591,4,Heartland Bank and Trust Company,https://adcentral.indeed.com/crm/history?aid=1...


## Use PLUS to get the CS & Sales Rep IDs for each affected advertiser
* Uses a nifty function that Boris taught me by feeding in column names into plus call
- Appends the *service_rep* and *sales_rep* columns to the existing dataframe

In [82]:
for col in ['service_rep','sales_rep']:
    affected_Accounts[col] = plus.get_advertiser_info(affected_Accounts.advid, intype = "advid", outtype = col)

In [71]:
affected_Accounts.tail()

,advid,,_advid,Link to AdCard,service_rep,sales_rep
161,8110093,1,NAPA Transportation,https://adcentral.indeed.com/crm/history?aid=8...,3914,950
162,8124395,18,mymuesli GmbH,https://adcentral.indeed.com/crm/history?aid=8...,5605,4579
163,8239733,10,“Campaign 2” Luxoft Global Operations GmbH,https://adcentral.indeed.com/crm/history?aid=8...,2806,2838
164,8301492,2,株式会社ヒト・コミュニケーションズ IB営業部,https://adcentral.indeed.com/crm/history?aid=8...,2331,1903
165,8442209,3,MaidPro Marin,https://adcentral.indeed.com/crm/history?aid=8...,4225,1


In [61]:
# just checking to see what the actual column names were for the get_user_info end point
rep_info.columns

Index([u'active', u'email', u'firstname', u'lastname', u'username'], dtype='object')

## Get the Rep Names, Email address & Append to df

* Again, use the column names being fed from the current df into a plus call
    - Once all data has been retreived, creates a new column *Full Name* which is just combining the first and last names for each rep
    - Appends the *Full Name* & *Email* columns to the df

In [83]:
for rep_type in ['service_rep', 'sales_rep']:
    PLUS = plus.get_user_info(affected_Accounts[rep_type]).reset_index()
    PLUS["Full Name"] = PLUS.firstname + " " + PLUS.lastname
    PLUS["Email"] = PLUS.email
    column_position = affected_Accounts.columns.values.tolist().index(rep_type) + 1
    affected_Accounts.insert(column_position, rep_type + " Full Name", PLUS["Full Name"])
    affected_Accounts.insert(column_position, rep_type + " Email", PLUS["Email"])

In [85]:
affected_Accounts.tail()

,advid,,_advid,Link to AdCard,service_rep,service_rep Email,service_rep Full Name,sales_rep,sales_rep Email,sales_rep Full Name
161,8110093,1,NAPA Transportation,https://adcentral.indeed.com/crm/history?aid=8...,3914,agomez@indeed.com,Amanda Gomez,950,matto@indeed.com,Matthew Owens
162,8124395,18,mymuesli GmbH,https://adcentral.indeed.com/crm/history?aid=8...,5605,svea@indeed.com,Svea von Gagern,4579,sszalaty@indeed.com,Sandra Szalaty
163,8239733,10,“Campaign 2” Luxoft Global Operations GmbH,https://adcentral.indeed.com/crm/history?aid=8...,2806,katya@indeed.com,Katya Sitko,2838,maxim@indeed.com,Maxim Staricov
164,8301492,2,株式会社ヒト・コミュニケーションズ IB営業部,https://adcentral.indeed.com/crm/history?aid=8...,2331,sho@indeed.com,Sho Sakurada,1903,gaku@indeed.com,Gaku Takeuchi
165,8442209,3,MaidPro Marin,https://adcentral.indeed.com/crm/history?aid=8...,4225,alinap@indeed.com,Alina Presbury,1,,Not Applicable


## Drop the service rep and sales rep ID fields, replace empty values with "-" and rename the company
- Export the final output to CSV and change the encoding to "utf-8" because of the special characters in the advertiser name

In [104]:
a = affected_Accounts.drop(affected_Accounts.columns[[4, 7]], axis = 1).replace("", "-").rename(columns = {"_advid" : "Company Name"})

In [108]:
a.to_csv("list_of_advids.csv", encoding = 'utf-8')